# Uploading Factor Risk Models

The GS Quant `FactorRiskModel` class gives users the power to upload their own risk models to Marquee for seamless integration with the Marquee Portfolio Analytics and Plot Tool Pro suite. After uploading a custom `FactorRiskModel`, users can access their factor model data programmatically using GS Quant, visualize their factor risk model data with Plot Tool Pro, or run historical factor attribution analysis on equity portfolios through the lens of their uploaded factor risk model with GS Quant's `Portfolio` class.

## Step 1: Authenticate and Initialize Your Session

First you will import the necessary modules and add your client id and client secret.

In [ ]:
import warnings

from gs_quant.session import GsSession, Environment
from gs_quant.models.risk_model import (
    FactorRiskModel,
    RiskModelCalendar,
    Term,
    CoverageType,
    UniverseIdentifier,
    FactorType,
    RiskModelFactor,
)
import datetime as dt


client = None
secret = None

## External users must fill in their client ID and secret below and comment out the lines below

# client = 'ENTER CLIENT ID'
# secret = 'ENTER CLIENT SECRET'

GsSession.use(Environment.PROD, client_id=client, client_secret=secret)
warnings.filterwarnings("ignore", category=RuntimeWarning)

print('GS Session initialized.')

## Step 2: Create a Factor Model

Input fields to create the initial Factor Risk Model object

| Attribute                        | Can be Modified? | Description                                                                                                                                                                                                                                                       
|----------------------------------|------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
| id                               | No               | Model ID                                                                                                                                                                                                                                                          |
| name                             | Yes              | Name of model                                                                                                                                                                                                                                                     |
| description                      | Yes              | Longer description of model                                                                                                                                                                                                                                       |
| term                             | Yes              | Term or horizon of model. One of: Long, Medium, Short, Trading, Daily                                                                                                                                                                                             |
| coverage                         | Yes              | Geographical coverage of assets within model universe. One of: Global, Region, Region Excluding Countries, Country                                                                                                                                                |
| vendor                           | Yes              | Who creates the model                                                                                                                                                                                                                                             |
| version                          | Yes              | Version of model                                                                                                                                                                                                                                                  |
| universe_identifier              | No               | Identifier used to upload the model's asset universe. One of: sedol, cusip, isin, bcid, gsid                                                                                                                                                                      |
| expected_update_time  (optional) | Yes              | Time (in UTC) data is expected to be published.                                                                                                                                                                                                                   |
| entitlements (optional)          | Yes              | Who can view, execute (display access in Marquee), query (via API), or upload data to the risk model. By default, the application that creates the risk model is granted admin entitlements, which include all these rights and the ability to edit entitlements. |


In [ ]:
model_id = 'VENDOR_COVERAGE_MODELNAME_TERM_VERSION'
model_name = 'My Risk Model'
description = 'My Custom Factor Risk Model'
term = Term.Medium
coverage = CoverageType.Country
universe_identifier = UniverseIdentifier.sedol
expected_update_time = dt.time(6, 0, 0)
vendor = 'Goldman Sachs'

Create the Factor Risk Model

In [ ]:
model = FactorRiskModel(
    id_=model_id,
    name=model_name,
    description=description,
    coverage=coverage,
    term=term,
    universe_identifier=universe_identifier,
    vendor=vendor,
    expected_update_time=expected_update_time,
    version=1,
)

model.save()
print(f"Successfully create a factor risk model with ID: {model.id}")

### A note on Model IDs

Model IDs are unique identifiers in Marquee that adhere to a standardized format, ensuring clarity and consistency for clients. The format is:

`VENDOR_COVERAGE_MODELNAME_TERM_VERSION`

- **Vendor**: The name of the entity or organization creating the model.
- **Coverage**: The geographical asset coverage of the model (e.g., "GLOBAL", "APAC", "EU", "US", "UK", "EM" (or Emerging), "DM" (for Developed Markets), "WW" (for World Wide) etc).
- **Term**: 1-3 character(s) denoting the time horizon of the model (e.g., "L", "S", "M", "TRD" for "Long", "Short", "Medium", or "Trading" respectively). Variations of these can also take the form of "MH" or "MT" for Medium Horizon and Medium term. 
- **Version**: The version number of the model, starting from 1 and incrementing with updates.

This format is designed to be intuitive and easily recognizable for clients, making it simple to identify the purpose and scope of a model at a glance.

## Step 3: Upload a Calendar To Your Model

The calendar associated with the Factor Risk Model contains the dates which the risk model should have posted data on to be considered "complete." The calendar can go further back as well as forward in time than the data that is currently posted for the calendar, but there cannot be any gaps in the data posted to the risk model according to the calendar. Please note that the `upload_calendar` function overwrites the previous calendar if one exists, so be sure to include all dates you want in the calendar.

In [ ]:
calendar = RiskModelCalendar(
    [
        '2021-01-29',
        '2021-01-28',
        '2021-01-27',
        '2021-01-26',
        '2021-01-25',
        '2021-01-22',
        '2021-01-21',
        '2021-01-20',
        '2021-01-19',
        '2021-01-18',
        '2021-01-15',
        '2021-01-14',
        '2021-01-13',
        '2021-01-12',
        '2021-01-11',
        '2021-01-08',
        '2021-01-07',
        '2021-01-06',
        '2021-01-05',
        '2021-01-04',
        '2021-01-01',
    ]
)

model.upload_calendar(calendar)

### How to update a calendar? 
To update a calendar, you can use the same `upload_calendar` method with a new list of dates. The new list will replace the existing calendar. If you want to add new dates without removing the existing ones, you must first retrieve the current calendar using `model.get_calendar()` and then append the new dates before uploading.


In [ ]:
# Get the existing calendar
existing_calendar = model.get_calendar().business_dates

# Append new dates
new_dates = ['2021-01-30', '2021-01-31']
updated_business_dates = list(existing_calendar) + new_dates

model.upload_calendar(RiskModelCalendar(updated_business_dates))

## Step 4: Upload Data To Your Model

Once the calendar is posted for a model, we can start uploading data to it.

The data must be uploaded in a specific format, which is defined in the `RiskModelData` class. The data must be uploaded for each date in the calendar. 

Below are the components of RiskModelData

#### 1. Factor Data


| Field            | Description                                                                                                                                             
|------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------
| factorId         | Unique ID of the factor (must be < 20 characters). It must map consistently to the same factor across every date.                                       |
| factorName       | Name of the factor. It can be any string and should be consistent across every date.                                                                    |
| factorCategoryId | Unique Id of the category that the factor belongs to (must be < 20 characters). It must map consistently to the same factor category across every date. |
| factorCategory   | Name of the category that the factor belongs to (Style, Industry, Market, Currency, etc.).                                                              |
| factorReturn     | Daily return of the factor in percent units (i.e. for 10%, use 10).                                                                                     |

In [ ]:
factor_data = [
    {
        "factorId": "1",
        "factorName": "Factor 1",
        "factorCategoryId": "RI",
        "factorCategory": "Style",
        "factorReturn": 0.39,
    },
    {
        "factorId": "2",
        "factorName": "Factor 2",
        "factorCategoryId": "RI",
        "factorCategory": "Style",
        "factorReturn": 1.99,
    },
    {
        "factorId": "3",
        "factorName": "Factor 3",
        "factorCategoryId": "RI",
        "factorCategory": "Style",
        "factorReturn": 0.29,
    },
    {
        "factorId": "4",
        "factorName": "Factor 4",
        "factorCategoryId": "MKT",
        "factorCategory": "Market",
        "factorReturn": -0.9,
    },
    {
        "factorId": "5",
        "factorName": "Factor 5",
        "factorCategoryId": "IND",
        "factorCategory": "Industry",
        "factorReturn": 0.2,
    },
]

#### 2. Asset Data

The asset data can be categorized into different components: 
   - **Factor Exposures**: factor loadings/zscores of assets in the universe.
   - **Asset risk data**: Total risk, specific risk and specific return
   - **Asset betas**: Historical beta and model-predicted beta
   - **Market data**: Daily return, price, market capitalization, trading volumes, and dividend yields
   
   
| Field                    | Optional? | Description                                                                                                                                                                                                   
|--------------------------|-----------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
| universe                 | No        | The model universe uploaded as an array of identifiers. The identifier type must match the model's universe identifier  |
| factorExposure           | No        | Array of dictionaries of factorId to the factor exposure of each asset in the universe.  There is no need to include a factorId in the map if the asset has zero exposure to the factor.|
| specificRisk             | No        | Array of annualized specific risk. Represented in percent units (i.e. for 10%, use 10). The order in this array must match the order of the universe array. |
| totalRisk                | No        | Array of annualized total risk. Represented in percent units (e.g., for 10%, use 10). The order in this array must match the order of the universe array. |
| specificReturn           | No        | Array of specific returns. Assign `None` for missing or N/A values. The order of this array must match the order of the universe array.|
| estimationUniverseWeight | No        | Array of weights in the estimation universe. The order of this array must match the order of the universe array. Assign `None` if an asset is not in the estimation universe.|
| predictedBeta            | No        | Array of predicted betas. Assign `None` for missing or N/A values. The order of this array must match the order of the universe array.|
| historicalBeta           | No        | Array of historical betas. Assign `None` for missing or N/A values. The order of this array must match the order of the universe array.|
| globalPredictedBeta      | Yes       | Array of predicted betas relative to a global market. This measure is not applicable to all models. Assign `None` for missing or N/A values. The order of this array must match the order of the universe array. |
| dailyReturn              | Yes        | Array of asset daily returns. Represented in percent units (e.g., for 10%, use 10). Assign `None` for missing or N/A values. The order in this array must match the order of the universe array. |
| price                    | Yes        | Array of asset prices. Assign `None` for missing or N/A values. The order in this array must match the order of the universe array. |
| currency                 | Yes        | Array of currencies. Assign `None` for missing or N/A values. The order in this array must match the order of the universe array.|
| capitalization           | Yes        | Array of asset market capitalizations in trading currency. Assign `None` for missing or N/A values. The order in this array must match the order of the universe array. |
| dividendYield            | Yes        | Array of dividend yields. Assign `None` for missing or N/A values. The ordering of this array must correspond to the ordering of the universe array |
| tradingVolume            | Yes        | Array of asset daily trading volumes. Assign `None` for missing or N/A values. The ordering of this array must correspond to the ordering of the universe array |


In [ ]:
asset_data = {
    "universe": ["ASSET1", "ASSET2", "ASSET3", "ASSET4", "ASSET5"],
    "factorExposure": [
        {"1": 0.48760401340992243, "2": -0.590034096365603},
        {"3": -0.8017494714963063, "4": 0.7862451506530919},
        {
            "5": -0.253403252938764,
        },
        {
            "1": 1.7,
            "2": 0.08459557023760955,
            "3": 1.8459557023760955,
            "4": 0.8459557023760955,
            "5": -1.8459557023760955,
        },
        {"2": -0.607987433173778},
    ],
    "totalRisk": [14.324714301179963, 17.071151010504597, 13.874122072602503, 14.479219370106968, 15.6372307010737],
    "specificRisk": [6.267696111058748, 7.348880522500647, 7.866871501833286, 5.835214680084606, 11.76129276971361],
    "specificReturn": [
        0.5659048855060385,
        0.37595374172320617,
        -0.24473722135505294,
        -0.9616000320458231,
        -0.7699674832309915,
    ],
    "predictedBeta": [
        0.8376437537112529,
        0.8825284596124832,
        0.8658162312132138,
        1.1663832117848132,
        1.1638549355761407,
    ],
    "historicalBeta": [
        0.9699683237602925,
        1.184349884914826,
        1.1478458610614193,
        1.1073962331683949,
        0.8610028132979528,
    ],
    "estimationUniverseWeight": [None, 0.5, 0.2, 0.3, None],
    "dailyReturn": [
        0.36864638955606754,
        -0.16247541720716152,
        -0.14959565439528988,
        0.3449230613583272,
        0.4624063629113383,
    ],
    "currency": ["USD", "USD", "USD", "USD", "USD"],
    "price": [146.85268872446377, 112.66584268302705, 102.8189727997599, 122.91632409053749, 131.95674622727046],
    "capitalization": [2310528821.941124, 8391014828.775409, 7744577643.201652, 1873806225.329452, 6606600945.467762],
    "issuerMarketCap": [
        8694261819.837845,
        7063154593.331768,
        1761291245.7391672,
        4251457616.7812405,
        5264161805.272642,
    ],
    "dividendYield": [4.578537816528908, 3.7261774596135204, 3.4343027973124207, 3.413227950682553, 2.175273338412861],
}

#### 3. Covariance Matrix

The covariance matrix is represented as a 2D array, where each row corresponds to a factor, and the elements in the inner arrays represent the covariances (in daily variance units) between that factor and every other factor. The ordering of both the rows and columns aligns with the ordering of the factor data list in the payload. The first array below contains variance-covariance data for the factor with ID "1" and the first element is the variance of the factor itself. The second element is the covariance between the first and second factor, and so on. The covariance matrix must be a square matrix, meaning it has the same number of rows and columns.

In [ ]:
covariance_matrix = [
    [0.9864, 0.026, 0.0965, 0.067, 0.0502],
    [0.026, 0.6192, 0.0272, 0.0437, 0.0185],
    [0.0965, 0.0272, 0.8771, 0.0863, 0.0407],
    [0.067, 0.0437, 0.0863, 0.9589, 0.0333],
    [0.0502, 0.0185, 0.0407, 0.0333, 0.5216],
]

#### 4. Risk Free Rates 

The currency rates data is represented as an object containing information about currency exchange rates and risk-free rates. It includes the following components: `currency`, an array of strings representing the currency tickers; `exchangeRate`, an array of daily USD exchange rates, where the ordering corresponds to the ordering of the currencies; and `riskFreeRate`, an array of annualized monthly risk-free rates (in percentage), also ordered to match the currencies. If the model has a numeraire different from USD, the exchangeRate will be an array of daily exchange rates in that currency

- currency: Array of currency tickers
- exchangeRate: Array of daily USD (or the risk model numeraire if not USD) exchange rates, ordered to match the currencies
- riskFreeRate: Array of annualized monthly risk-free rates (in percentage), ordered to match the currencies

In [ ]:
currency_data = {
    'currency': ['USD', 'EUR', 'GBP'],
    'exchangeRate': [1.0, 0.85, 0.75],
    'riskFreeRate': [0.01, 0.005, 0.007],
}

#### 5. Issuer Specific Covariance

The issuer-specific covariance data is represented as an object that captures the covariance between two assets in daily variance units. It consists of three components: universeId1, an array of strings representing the first set of universe identifiers, which must exist in the asset universe; universeId2, an array of strings representing the second set of universe identifiers, also required to exist in the asset universe; and covariance, an array of numbers where each value represents the covariance between the corresponding identifiers at the same index in universeId1 and universeId2.

- universeId1: Array of assets with issuer specific covariance to the asset in universeId2 at the same index. Each asset must also be present in the Asset Data universe
- universeId1: Array of assets with issuer specific covariance to the asset in universeId1 at the same index. Each asset must also be present in the Asset Data universe
- covariance: Array of the covariance between universeId1 and universeId2 at the same index. In daily variance units

In [ ]:
issuer_specific_covariance = {
    'universeId1': ['ASSET1', 'ASSET2'],
    'universeId2': ['ASSET3', 'ASSET5'],
    'covariance': [0.03754, 0.01234],
}

### 6. Factor Portfolios

The factor portfolios data is represented as an object that defines a set of assets combining to form a portfolio representing a single factor. This portfolio has unit exposure to the specified factor and zero exposure to all other factors, making it useful for mapping intraday factor return data. It consists of two components: `universe`, an array of strings representing the universe of the factor portfolios, which must be a subset of the model asset universe; and `portfolio`, an array of dictionaries containing factor identifiers and their corresponding portfolio weights.

The portfolios array contains objects with the following fields:
- factorId: The ID of the factor corresponding to the factor data's factorIds
- weights: An array of weights for each asset ID in the universe, corresponding to the ordering of the universe. The weights must sum to 1 and can include zero values (null values are not allowed).

In [ ]:
factor_portfolios = {
    "universe": ["ASSET2", "ASSET3", "ASSET4"],
    "portfolio": [
        {"factorId": "1", "weights": [0.3545899480088869, 0.2880743474246568, 0.35733570456645636]},
        {"factorId": "2", "weights": [0.4390053364378467, 0.2907941789867976, 0.2702004845753557]},
        {"factorId": "3", "weights": [0.3547375024699844, 0.29918415302858037, 0.3460783445014351]},
        {"factorId": "4", "weights": [0.5821881487398893, 0.06066318669175055, 0.35714866456836014]},
        {"factorId": "5", "weights": [0.4078609724431924, 0.39878265902389126, 0.1933563685329163]},
    ],
}




### Upload Data

Now we are ready to upload risk model data. Note you can only upload on dates that are in the model's calendar.
The data must be uploaded in a specific format, which is defined in the `RiskModelData` class. The data must be uploaded for each date in the calendar, one date at a time.


In [ ]:
data = {
    'date': '2021-01-13',
    'factorData': factor_data,
    'covarianceMatrix': covariance_matrix,
    'assetData': asset_data,
    'currencyRatesData': currency_data,
    'issuerSpecificCovariance': issuer_specific_covariance,
    'factorPortfolios': factor_portfolios,
}

risk_model = FactorRiskModel.get(model_id)
risk_model.upload_data(data)

#### Important

For large payloads, use the `max_asset_batch_size` parameter to split the data into smaller batches. This is particularly important for large models (asset universe larger than 10,000), as it helps prevent timeouts during the upload process. Typically, a batch_size of `10000` works well. 


In [ ]:
risk_model.upload_data(data, max_asset_batch_size=1000)

## Step 5: Check which days have data posted

The `get_dates` method returns a list of dates for which data has been posted to the risk model. This is useful for verifying whether the data upload was successful. Upon completing the historical data backfill, all dates returned by this method should match the dates in the model's calendar. If any dates are missing, it may indicate that the data upload was incomplete or has not occurred yet.


In [ ]:
dates_posted = risk_model.get_dates()
print(dates_posted)

## Step 6: Uploading Partial data
The `upload_data` method can be used to upload partial data, which is useful for updating subsets of data without re-uploading the entire dataset. Data that can be uploaded partially includes `factorData` and `covarianceMatrix`, `assetData`, `factorPortfolios`, and `issuerSpecificCovariance`. Note that `factorData` and `covarianceMatrix` must already be uploaded when uploading partial data. If not, you must upload them first. The partial data must conform to the same structure as the full data upload but can represent a subset of the complete dataset.



In [ ]:
# Uploading factor portfolios separately
factor_portfolios = {'date': '2021-01-13', 'factorPortfolios': factor_portfolios}

risk_model.upload_data(factor_portfolios, max_asset_batch_size=1000)

# Uploading issuer specific covariance separately
issuer_specific_covariance = {'date': '2021-01-13', 'issuerSpecificCovariance': issuer_specific_covariance}
risk_model.upload_data(issuer_specific_covariance, max_asset_batch_size=1000)

## Step 7: Enhance Factor Descriptions and Tool Tips

The last step is adding tooltips and descriptions to the risk model factors. We highly encourage you to do this for every non-binary factor in your model (such as style factors) so that Marquee UI users of your model can leverage the tooltips and descriptions to better understand how the factors were constructed and what they represent. The snippet below should be repeated for each factor for which you are updating descriptions and tooltips. The `identifier` field must match the `factorId` in the factor data you uploaded earlier.

In [ ]:
identifier = '3'
tooltip = 'Short description that appears when you hover over the factor name on our UI.'
description = 'Longer description that appears on the portfolio drill-down page of this factor.'
glossary_description = 'Longest description to describe the factor in depth on our risk model glossary page.'

factor = RiskModelFactor(
    identifier=identifier,
    type_=FactorType.Factor,
    tooltip=tooltip,
    description=description,
    glossary_description=glossary_description,
)

risk_model.save_factor_metadata(factor)